In [2]:
import copy
import glob
import random
import regex as re
import numpy as np
import pandas as pd
import datetime as dt
import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline
%load_ext rpy2.ipython
%run ../airquality/data/gen_daily_targets.py
%run ../airquality/data/prepare_data.py

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
pred_svm = pd.read_csv('../reports/pred_ssvm_Jonas_8_1.csv', index_col=0)
pred_svm.head()

,ES0691A,ES1396A,ES1438A,ES1480A,ES1679A,ES1856A,ES1992A,date
0,0.16,0.16,0.10,0.14,0.17,0.34,0.31,2013-01-01
1,0.16,0.16,0.10,0.14,0.17,0.34,0.31,2013-01-02
2,0.16,0.16,0.10,0.14,0.17,0.34,0.31,2013-01-03
3,0.16,0.16,0.10,0.14,0.17,0.34,0.31,2013-01-04
4,0.16,0.16,0.10,0.14,0.17,0.34,0.31,2013-01-05


In [5]:
pred_knn = pd.read_csv('../reports/pred_knn_Jonas_7_3.csv', index_col=0)
pred_knn.head()

,ES0691A,ES1396A,ES1438A,ES1480A,ES1679A,ES1856A,ES1992A,date
0,0.30,0.14,0.09,0.17,0.07,0.58,0.68,2013-01-01
1,0.30,0.19,0.09,0.22,0.13,0.62,0.64,2013-01-02
2,0.30,0.19,0.17,0.22,0.13,0.71,0.68,2013-01-03
3,0.45,0.14,0.22,0.22,0.13,0.83,0.68,2013-01-04
4,0.55,0.14,0.17,0.22,0.13,0.88,0.64,2013-01-05
